Vamos a utilizar CryptoWatch, un portal que nos facilita una API para la obtencion de datos de cryptomonedas.

In [ ]:
!pip install cryptowatch-sdk
!pip install google-cloud-aiplatform
!pip install sktime
!mkdir -p $HOME/.cw
!echo "apikey: XNS40GIEET88LR35UE2S" > $HOME/.cw/credentials.yml
!cat  $HOME/.cw/credentials.yml

  Using cached numpy-1.19.3-cp37-cp37m-manylinux2010_x86_64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 2.3.6 requires scikit-learn==0.23.2, but you have scikit-learn 0.24.2 which is incompatible.
pandas-gbq 0.13.3 requires google-cloud-bigquery[bqstorage,pandas]<2.0.0dev,>=1.11.1, but you have google-cloud-bigquery 2.18.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.25.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


apikey: XNS40GIEET88LR35UE2S


In [ ]:
import cryptowatch as cw
import pandas as pd
from datetime import datetime, timedelta
#from google.cloud import bigquery
#from google.cloud import aiplatform
from matplotlib import pyplot
import logging
from sktime.utils.plotting import plot_series

Reducimos el Verbose

In [ ]:
logging.basicConfig()
logging.getLogger("cryptowatch").setLevel(logging.WARNING)

Para la parte de Google Cloud definimos ya algunas variables

In [ ]:
MARKET="coinbase-pro" #Usaremos el mercado del conocido portal Coinbase
PROJECT_ID = "eliaslopez-test" 
REGION = "us-central1" 
DATASET = "cryto_data"
TEMP_BUCKET="master_bucket_uscentral1"

Vamos a realizar una busqueda de todas las Cryptomonedas que han obtenido un crecimiento mayor al 15% en la ultima semana, para elegir 1 de ellas.

In [ ]:
#Si quisieramos realizar un analisis de las Cryptos mas rentables de la ultima semana:
umbral_benef = 20 #Establecemos un rendimiento del 15% en las ultimas 24h

coinbase = cw.markets.list(MARKET)
for market in coinbase.markets:

    try:
        ticker = "{}:{}".format(market.exchange, market.pair).upper()
        candles = cw.markets.get(ticker, ohlc=True, periods=["1w"]) 
#Establecemos el periodo en 1 semana atras


        close_ts, wkly_open, wkly_close = (
            candles.of_1w[-1][0],
            candles.of_1w[-1][1],
            candles.of_1w[-1][4],
        )

        if wkly_open == 0:
            continue
        perf = (wkly_open - wkly_close) * 100 / wkly_open

    
        if perf >= umbral_benef:
            open_ts = datetime.utcfromtimestamp(close_ts) - timedelta(days=7)
            print("{} ganó {:.2f}% desde {}".format(ticker, perf, open_ts))
    except:
        print("Execpcion capturada, pero continuamos ..")

Execpcion capturada, pero continuamos ..
Execpcion capturada, pero continuamos ..
COINBASE-PRO:LPTUSD ganó 21.89% desde 2022-02-10 00:00:00
Execpcion capturada, pero continuamos ..
COINBASE-PRO:ACHUSD ganó 20.42% desde 2022-02-10 00:00:00
Execpcion capturada, pero continuamos ..
COINBASE-PRO:AGLDUSDT ganó 22.38% desde 2022-02-10 00:00:00
COINBASE-PRO:AGLDUSD ganó 22.38% desde 2022-02-10 00:00:00
Execpcion capturada, pero continuamos ..
Execpcion capturada, pero continuamos ..
Execpcion capturada, pero continuamos ..
COINBASE-PRO:JASMYUSD ganó 25.07% desde 2022-02-10 00:00:00
Execpcion capturada, pero continuamos ..
Execpcion capturada, pero continuamos ..
Execpcion capturada, pero continuamos ..
Execpcion capturada, pero continuamos ..
Execpcion capturada, pero continuamos ..
COINBASE-PRO:GFIUSD ganó 22.28% desde 2022-02-10 00:00:00
Execpcion capturada, pero continuamos ..
Execpcion capturada, pero continuamos ..
Execpcion capturada, pero continuamos ..
Execpcion capturada, pero contin

In [ ]:
#Elegimos 1 cryptomoneda, en nuestro caso utilizaremos el Bitcoin en dolares (USD)
TICKET="BTCUSD"
TABLE = "candles_15min_"+TICKET
candles = cw.markets.get(MARKET+":"+TICKET, ohlc=True)

In [ ]:
#Creamos un dataframe con la Crypto elegida
rows_list = []
#En la linea siguiente stablecemos que la informacion sea cada 30 minutos
for x in candles.of_15m:
    close_ts = datetime.utcfromtimestamp(x[0])
    open_value = x[1]
    high_value = x[2]
    low_value = x[3]
    close_value = x[4]
    volume_base = x[5]
    volume_quote = x[6]
    rows_list.append([TICKET,close_ts , open_value , high_value , low_value ,close_value ,volume_base ,volume_quote])
df = pd.DataFrame(rows_list,columns = ["ticket","close_ts" , "open_value" , "high_value" , "low_value" ,"close_value" ,"volume_base" ,"volume_quote" ])
df.head()

,ticket,close_ts,open_value,high_value,low_value,close_value,volume_base,volume_quote
0,BTCUSD,2022-02-03 07:00:00,36913.47,36977.72,36906.81,36951.59,34.139094,1.261280e+06
1,BTCUSD,2022-02-03 07:15:00,36951.59,37068.25,36941.99,37051.50,47.032167,1.740453e+06
2,BTCUSD,2022-02-03 07:30:00,37051.52,37058.90,37003.58,37046.55,25.410838,9.409487e+05
3,BTCUSD,2022-02-03 07:45:00,37045.85,37079.30,36975.74,37062.99,38.765995,1.435138e+06
4,BTCUSD,2022-02-03 08:00:00,37062.99,37118.61,37039.49,37045.41,24.902655,9.232033e+05


# Vamos a utilizar el Framework pyCaret, el cual nos permite comparar entre los mejores modelos de regresion

In [ ]:
!pip install pyyaml
!pip install pycaret-ts-alpha

In [ ]:
import time
import numpy as np
import pandas as pd

from pycaret.datasets import get_data

from pycaret.internal.pycaret_experiment import TimeSeriesExperiment

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
from pandas.core.indexes.datetimes import date_range
index = pd.DatetimeIndex(df["close_ts"])
rng = date_range(index[0], periods = 1000, freq="15min")
data = df["close_value"].tp_numpy()
df_series = pd.Series(data=data, index=rng)

AttributeError: ignored